# Training Model Rekomendasi Film dengan TensorFlow

Notebook ini bertujuan untuk membuat dan melatih model rekomendasi film menggunakan TensorFlow/Keras. Model ini akan belajar dari data rating yang diberikan oleh pengguna untuk memprediksi rating film yang belum mereka tonton.

## 1. Import Library

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

## 2. Load Dataset

In [ ]:
ratings_df = pd.read_csv('data/ratings.csv')
movies_df = pd.read_csv('data/movies.csv')

print("Contoh data ratings:")
print(ratings_df.head())
print("\nJumlah data ratings:", len(ratings_df))

## 3. Pra-pemrosesan Data

Model Jaringan Saraf Tiruan, khususnya Embedding Layer, memerlukan input berupa integer yang berurutan (misal: dari 0 hingga jumlah unik user/movie - 1). Oleh karena itu, kita perlu mengubah `userId` dan `movieId` yang asli menjadi indeks integer.

In [ ]:
# Menggunakan LabelEncoder untuk mengubah userId dan movieId menjadi indeks integer
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

ratings_df['user_idx'] = user_encoder.fit_transform(ratings_df['userId'])
ratings_df['movie_idx'] = movie_encoder.fit_transform(ratings_df['movieId'])

num_users = len(ratings_df['user_idx'].unique())
num_movies = len(ratings_df['movie_idx'].unique())

print(f"Jumlah user unik: {num_users}")
print(f"Jumlah movie unik: {num_movies}")
print("\nData setelah di-encode:")
print(ratings_df.head())

## 4. Split Data

Kita akan membagi dataset menjadi data training (80%) dan data testing (20%).

In [ ]:
X = ratings_df[['user_idx', 'movie_idx']].values
y = ratings_df['rating'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 5. Membangun Model Jaringan Saraf Tiruan (Neural Network)

Arsitektur model ini meniru faktorisasi matriks:
1.  **Input Layers**: Dua input, satu untuk `user_idx` dan satu untuk `movie_idx`.
2.  **Embedding Layers**: Setiap input akan dipetakan ke sebuah vektor padat (embedding). Lapisan ini yang akan mempelajari representasi laten dari setiap user dan movie.
3.  **Dot Product**: Vektor embedding user dan movie akan digabungkan menggunakan operasi *dot product* untuk menghasilkan prediksi rating.

In [ ]:
embedding_dim = 50 # Dimensi vektor embedding, bisa di-tune

# Input untuk user
user_input = keras.Input(shape=(1,), name='user_input')
user_embedding = layers.Embedding(num_users, embedding_dim, name='user_embedding')(user_input)
user_vec = layers.Flatten(name='flatten_user')(user_embedding)

# Input untuk movie
movie_input = keras.Input(shape=(1,), name='movie_input')
movie_embedding = layers.Embedding(num_movies, embedding_dim, name='movie_embedding')(movie_input)
movie_vec = layers.Flatten(name='flatten_movie')(movie_embedding)

# Dot product dari kedua embedding vector
dot_product = layers.Dot(axes=1, name='dot_product')([user_vec, movie_vec])

# Membuat dan melihat ringkasan model
model = keras.Model([user_input, movie_input], dot_product)
model.summary()

## 6. Kompilasi dan Training Model

- **Optimizer**: `adam` adalah pilihan yang umum dan bagus.
- **Loss Function**: `mean_squared_error` (MSE) cocok untuk masalah regresi seperti memprediksi rating.

Hyperparameter seperti `epochs` dan `batch_size` dipilih agar proses training tidak terlalu lama dan efisien di GPU.

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

# Memisahkan input untuk training
X_train_user = X_train[:, 0]
X_train_movie = X_train[:, 1]

# Memisahkan input untuk testing
X_test_user = X_test[:, 0]
X_test_movie = X_test[:, 1]

history = model.fit(
    [X_train_user, X_train_movie],
    y_train,
    batch_size=64, # Ukuran batch yang cukup baik untuk GPU
    epochs=5,      # Jumlah epoch yang tidak terlalu banyak untuk awal
    validation_data=([X_test_user, X_test_movie], y_test)
)

## 7. Evaluasi Model

Kita akan mengevaluasi model pada data test untuk melihat seberapa baik performanya. Metrik yang umum digunakan adalah Root Mean Squared Error (RMSE).

In [ ]:
loss = model.evaluate([X_test_user, X_test_movie], y_test)
rmse = np.sqrt(loss)
print(f"Test Loss (MSE): {loss:.4f}")
print(f"Test RMSE: {rmse:.4f}")

## 8. Simpan Model

Menyimpan model yang sudah dilatih agar bisa digunakan kembali tanpa perlu training ulang.

In [ ]:
os.makedirs("models", exist_ok=True)
model.save("models/tf_model.keras")
print("Model TensorFlow berhasil disimpan di models/tf_model.keras")

## 9. Membuat Fungsi Rekomendasi

In [ ]:
def recommend_movies_for_user_tf(user_id, model, movies_df, ratings_df, user_encoder, movie_encoder, n=10):
    # 1. Dapatkan semua movieId yang ada
    all_movie_ids = movies_df['movieId'].unique()
    
    # 2. Dapatkan movieId yang sudah ditonton oleh user
    watched_movie_ids = ratings_df[ratings_df['userId'] == user_id]['movieId'].unique()
    
    # 3. Dapatkan movieId yang belum ditonton
    unseen_movie_ids = [m for m in all_movie_ids if m not in watched_movie_ids]
    
    # Filter film yang belum ditonton agar hanya berisi film yang dikenal oleh encoder
    known_unseen_movie_ids = [m for m in unseen_movie_ids if m in movie_encoder.classes_]
    
    # 4. Siapkan data untuk prediksi
    user_idx = user_encoder.transform([user_id])[0]
    unseen_movie_idx = movie_encoder.transform(known_unseen_movie_ids)
    
    user_input_array = np.array([user_idx] * len(unseen_movie_idx))
    movie_input_array = np.array(unseen_movie_idx)
    
    # 5. Lakukan prediksi
    predictions = model.predict([user_input_array, movie_input_array]).flatten()
    
    # 6. Urutkan dan ambil top-N
    results_df = pd.DataFrame({
        'movieId': known_unseen_movie_ids,
        'predicted_rating': predictions
    })
    top_n_results = results_df.sort_values(by='predicted_rating', ascending=False).head(n)
    
    # 7. Gabungkan dengan judul film untuk ditampilkan
    recommendations = movies_df.merge(top_n_results, on='movieId')
    
    return recommendations[['movieId', 'title', 'predicted_rating']]

## 10. Coba Rekomendasi

In [ ]:
# Anda bisa memuat model lagi jika kernel di-restart
# model = keras.models.load_model("models/tf_model.keras")

# Pilih user ID untuk mendapatkan rekomendasi
test_user_id = 33 

# Dapatkan rekomendasi
recommendations = recommend_movies_for_user_tf(
    test_user_id, model, movies_df, ratings_df, user_encoder, movie_encoder, n=10
)

print(f"Rekomendasi film untuk User ID: {test_user_id}")
print(recommendations)